In [52]:
# Importo il modulo requests
import threading
import requests
import toml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import math
import more_itertools 
from progress.bar import IncrementalBar
import re

import plotly.express as px

In [53]:
# Load the TOML data into a Python dictionary
with open("config.toml") as f:
    data = toml.load(f)

api_key = data['api_key']
base_bing_url = data['bing_url']
dimension_searc_area = data['dimension_searc_area']
search_precision = data['search_precision']
num_threads = data['num_threads']

region = 'Calabria'

In [54]:
def radiants_distance(lat1, lon1, lat2, lon2):
    '''return differenze from point in latitude and longitude'''
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Applicare la formula dell'averseno per calcolare la distanza angolare
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    return c


def get_rnd_point_in_limits(lat_c, lon_c, raggio,
                            min_lat=-math.pi / 2 , max_lat=math.pi / 2,
                            min_lon=-math.pi,max_lon=math.pi, n=10):
    '''return n random point in limit defined as input parameter '''
    punti = []

    # Iterare fino a che non si raggiunge il numero desiderato di punti
    while len(punti) < n:
        # Generare una latitudine e una longitudine casuali tra -$\pi$/2 e $\pi$/2 radianti
        lat_p = random.uniform(min_lat, max_lat)
        lon_p = random.uniform(min_lon, max_lon)

        # Calcolare la distanza tra il punto casuale e il centro dell'area circolare
        distanza = radiants_distance(lat_c, lon_c, lat_p, lon_p)

        # Se la distanza è minore o uguale al raggio, aggiungere il punto alla lista dei punti
        if distanza <= raggio:
            punti.append((math.degrees(lat_p), math.degrees(lon_p)))

    # Restituire la lista dei punti in gradi
    return punti


def get_rnd_point_on_circle_and_print_map(lat_c, lon_c, raggio, n = 100):
    '''return n random point on a circle defined by latitude, longitude and radius (in radiants)'''
    # Creare una lista vuota per contenere i punti
    punti = []
    punti_lat_perimeto_cerchio = []
    punti_lon_perimeto_cerchio = []

    n_punti_perimetro = 100

    # Calcolare l'angolo tra ogni punto in radianti
    angolo = 2 * math.pi / n_punti_perimetro

    # Iterare su ogni punto
    for i in range(n_punti_perimetro):
        # Calcolare la latitudine e la longitudine del punto in radianti
        lat_p = math.asin(math.sin(lat_c) * math.cos(raggio) + math.cos(lat_c) * math.sin(raggio) * math.cos(i * angolo))
        lon_p = lon_c + math.atan2(math.sin(i * angolo) * math.sin(raggio) * math.cos(lat_c), math.cos(raggio) - math.sin(lat_c) * math.sin(lat_p))
        punti_lat_perimeto_cerchio.append(lat_p)
        punti_lon_perimeto_cerchio.append(lon_p)
        # Convertire la latitudine e la longitudine del punto in gradi
        lat_p = math.degrees(lat_p)
        lon_p = math.degrees(lon_p)

        punti.append((lat_p, lon_p))

    min_lat = min(punti_lat_perimeto_cerchio)
    max_lat = max(punti_lat_perimeto_cerchio)
    min_lon = min(punti_lon_perimeto_cerchio)
    max_lon = max(punti_lon_perimeto_cerchio)

    punti_rnd = get_rnd_point_in_limits(lat_c, lon_c, raggio,min_lat, max_lat, min_lon, max_lon, n)
    for punto_rnd in punti_rnd:
        punti.append(punto_rnd)


    df_rnd = pd.DataFrame()
    df_rnd['lat'] = [l[0] for l in punti ]
    df_rnd['lon'] = [l[1] for l in punti ]

    fig2 = px.scatter_mapbox(df_rnd,
                        lat="lat",
                        lon="lon",
                        color_discrete_sequence=["blue"],
                        zoom=5,
                        height=500
                    )
    fig2.update_layout(mapbox_style="open-street-map")
    fig2.show()

    # Restituire la lista dei punti
    return punti

In [55]:
#TEST and view on map
user_km = 10

degree = user_km/111

#39.06768799, 16.34765053
degree
test = get_rnd_point_on_circle_and_print_map(math.radians(39.06768799),math.radians(16.34765053), math.radians(degree), 100)


In [56]:
user_km = 100

degree = user_km/111

degree

0.9009009009009009

In [57]:
def get_address_by_lat_and_long(api_key_bing, url, lat, lon):
    '''return address and postal code near point idenitified by latitude and longitude'''
    list_of_results = []
    url = f"{url}{lat},{lon}"

    # Definisco i parametri della richiesta inversa
    params = {
        "includeEntityTypes":  "Address",
        "includeNeighborhood": 1,
        "key": api_key_bing
    }

    # Invio la richiesta inversa e ottengo la risposta
    response = requests.get(url, params=params)
    
    # Controllo se la richiesta inversa è andata a buon fine
    if response.status_code == 200:
        data = None
        # Estraggo i dati JSON dalla risposta inversa
        try:
            data = response.json()
        except Exception as err:
            print(f'{err=}')
        
        data_status_code =  data["statusCode"]

        # Controllo se ci sono risultati validi
        if data_status_code == 200:
            
            # Estraggo tutti i risultati (i numeri civici)
            results = data["resourceSets"][0]["resources"]
            # Per ogni risultato, estraggo e stampo il numero civico e l'indirizzo completo
            for result in results:
                indirizzo_e_numero_civico = result["address"]["addressLine"]
                numero_civico = [int(s) for s in re.findall(r' \d+', indirizzo_e_numero_civico)]
                indirizzo_completo = result["address"]["formattedAddress"]
                regione = result["address"]["adminDistrict"]
                provincia = result["address"]["adminDistrict2"]
                comune = result["address"]["locality"]
                coordinate = result["geocodePoints"][0]["coordinates"]
                #print(result)
 
                list_of_results.append( ( regione, provincia, comune, indirizzo_completo, indirizzo_e_numero_civico, numero_civico, coordinate))
            
        else:
            # Stampo un messaggio di errore se non ci sono risultati validi
            print(f"Non sono stati trovati risultati validi, la richiesta alla mappe bing ha risposto con stato={data_status_code}")
    else:
        # Stampo un messaggio di errore se la richiesta inversa non è andata a buon fine
        print(f"Errore nella richiesta di recupero indirizzi: {response.status_code}")

    return list_of_results

In [58]:
def find_only_address_of_region_and_with_number(api_key_bing,url, points_for_request, region_ref, provinces, towns, address_complete,address_lite, latitudes, longitudes ):    
    with IncrementalBar('recupero indirizzi...', max=len(points_for_request)) as p_bar:
        for item_lat, item_lng in points_for_request:

            response_list = get_address_by_lat_and_long(api_key_bing,url, item_lat,  item_lng)

            for ( regione_resp,provincia_resp, comune_resp,  indirizzo_completo_resp, indirizzo_e_numero_civico_resp, numero_civico_resp, coordinate_resp)  in response_list:
                if indirizzo_completo_resp and regione_resp == region_ref and len(numero_civico_resp)>0 and numero_civico_resp[0] > 0 :
                    provinces.append(provincia_resp)
                    towns.append(comune_resp)
                    address_complete.append(indirizzo_completo_resp)
                    address_lite.append(indirizzo_e_numero_civico_resp)
                    latitudes.append(coordinate_resp[0])
                    longitudes.append(coordinate_resp[1])

            p_bar.next()

In [59]:
def recover_address_number_from_region_name ( api_key_bing, url,  region_name, radius = 10, precision=10, num_threads=2 ):
    '''retrieve ddress_number from a city name using bing maps api'''

    provinces = []
    towns = []
    address_complete = []
    address_lite = []
    latitudes = []
    longitueds = []

    # Definisco i parametri della richiesta
    params = {
        "adminDistrict": region_name,
        "key": api_key_bing
    }

    # Invio la richiesta e ottengo la risposta
    response = requests.get(url, params=params)

    # Controllo se la richiesta è andata a buon fine
    if response.status_code == 200:
        # Estraggo i dati JSON dalla risposta
        data = response.json()

        # Controllo se ci sono risultati validi
        if data["statusCode"] == 200:
            
            # Estraggo il primo risultato (il più rilevante)
            result = data["resourceSets"][0]["resources"][0]

            # Estraggo le coordinate geografiche del comune
            lat = result["point"]["coordinates"][0]
            lng = result["point"]["coordinates"][1]

            # Stampo le coordinate geografiche della regione
            print(f"Le coordinate geografiche di riferimento usate per {region_name} sono: {lat}, {lng}")
            points = []
            #recupero il admmin_district_ref della regione per filtare successivamente eventuali punti di altri regioni
            (regione, _,  _,  _, _, _, _ ) = get_address_by_lat_and_long(api_key_bing,url, lat, lng)[0]

            if regione:

                points = get_rnd_point_on_circle_and_print_map(math.radians(lat), math.radians(lng), math.radians(radius), precision)

               # find_only_address_of_region_and_with_number(api_key_bing,url, points, regione, provinces, towns, address_complete,address_lite, latitudes, longitueds )


                diveded_points = list(more_itertools.chunked(points, len(points) // num_threads))
                print(diveded_points)
                print(len(diveded_points))

                threads = list()

                for sub_points in diveded_points: 
                    t = threading.Thread(target=find_only_address_of_region_and_with_number,args=(api_key_bing,url, sub_points, regione, provinces, towns, address_complete,address_lite, latitudes, longitueds ))
                    threads.append(t)
                    t.start()
                
                for t in threads:
                    t.join()

            else:
                print(f"Fallito recupero informazioni in {region_name}")
                
        else:
            # Stampo un messaggio di errore se non ci sono risultati validi
            print(f"Nessun risultato trovato in {region_name}")
    else:
        # Stampo un messaggio di errore se la richiesta non è andata a buon fine
        print(f"Errore nella richiesta: {response.status_code}")

        
    df_address_region = pd.DataFrame()
    df_address_region['Provincia'] = provinces
    df_address_region['Comune'] = towns
    df_address_region['Indirizzo completo'] = address_complete
    df_address_region['Indirizzo'] = address_lite
    df_address_region['latitudine'] = latitudes
    df_address_region['longitudine'] = longitueds

    #df_address_region.drop_duplicates(inplace=True)

                            # Stampo il numero dei risultati trovati
    if len(address_complete)> 0:
        print(f"Trovati {len(address_complete)} numeri civici all'interno di un cerchio di raggio {radius} gradi con centro {region_name} ({lat}, {lng})")

    

    return df_address_region


In [60]:
df_region = recover_address_number_from_region_name(api_key,base_bing_url, region, dimension_searc_area, search_precision, num_threads)

Le coordinate geografiche di riferimento usate per Calabria sono: 39.06768799, 16.34765053


[[(39.167687990000005, 16.34765053), (39.16749038295821, 16.35574936228883), (39.166898345021075, 16.36381609627221), (39.16591422260938, 16.371818762479382), (39.164541915958765, 16.379725648565998), (39.162786863467176, 16.387505426522615), (39.16065601987407, 16.395127278265075), (39.15815782836277, 16.402561019079922), (39.15530218670354, 16.409777218408678), (39.152100407579475, 16.416747317467824), (39.148565173261645, 16.42344374321699), (39.144710484823165, 16.429840018205567), (39.14055160610441, 16.435910865848093), (39.136105002662994, 16.441632310700683), (39.13138827596223, 16.446981773334784), (39.12642009307118, 16.45193815943005), (39.121220112167016, 16.45648194273543), (39.11580890414657, 16.460595241575994), (39.11020787066959, 16.464261888612757), (39.10443915896915, 16.46746749359344), (39.0985255737768, 16.470199498863703), (39.092490486720926, 16.472447227440398), (39.086357743564974, 16.474201923481047), (39.08015156965989, 16.47545678501651), (39.07389647399099

In [33]:
df_region

,Provincia,Comune,Indirizzo completo,Indirizzo,latitudine,longitudine
0,Cosenza,Rogliano,"Contrada Cortivetere 20, 87054 Rogliano Cosenz...",Contrada Cortivetere 20,39.167688,16.347651
1,Cosenza,Rogliano,"Contrada Balzata 6, 87054 Rogliano Cosenza, Italy",Contrada Balzata 6,39.169063,16.364340
2,Cosenza,Rogliano,"Contrada Balzata 50, 87054 Rogliano Cosenza, I...",Contrada Balzata 50,39.166030,16.372652
3,Cosenza,Rogliano,"Contrada Balzata 84, 87054 Rogliano Cosenza, I...",Contrada Balzata 84,39.165225,16.378528
4,Cosenza,Parenti,"Contrada Margillera 7B, 87040 Parenti Cosenza,...",Contrada Margillera 7B,39.155974,16.413005
...,...,...,...,...,...,...
70,Cosenza,Colosimi,"Corso Vittorio Emanuele 88, 87050 Colosimi Cos...",Corso Vittorio Emanuele 88,39.118944,16.401690
71,Catanzaro,Soveria Mannelli,"Strada Statale 19 31, 88049 Soveria Mannelli C...",Strada Statale 19 31,39.057868,16.410098
72,Catanzaro,Lamezia Terme,"Contrada Serra Castagna 94, 88046 Lamezia Term...",Contrada Serra Castagna 94,39.012755,16.247659
73,Catanzaro,Carlopoli,"Via delle Ginestre 45, 88040 Carlopoli Catanza...",Via delle Ginestre 45,39.044390,16.447084


In [ ]:
df_region.to_csv(f"Indirizzi_{region}.csv", sep=';', index=False, encoding='utf-8' ) 